# RAG - Metrics & Evaluation

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2008%20-%20RAG_Metrics%26Evaluation.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

## RAG (and LLM) Metrics

Evaluating LLMs and retrieval-augmented generation (RAG) systems requires a detailed analysis of individual components and the system as a whole. Setting baseline values for pieces, such as chunking logic and embedding models, then assessing each part individually and end-to-end is critical for understanding the impact of changes on the system’s overall performance. Holistic modules in these evaluations don’t always require ground-truth labels, as they can be assessed based on the language model’s query, context, response, and interpretations.

Here are five commonly used metrics for evaluating your systems:

1.  [Correctness](https://docs.llamaindex.ai/en/latest/examples/evaluation/correctness_eval.html): This metric assesses whether the generated answer aligns with a given query’s reference answer. It requires labels and involves verifying the accuracy of the generated answer by comparing it to a predefined reference answer.
2.  [Faithfulness](https://docs.llamaindex.ai/en/latest/examples/evaluation/faithfulness_eval.html): This evaluates the integrity of the answer concerning the retrieved contexts. The faithfulness metric ensures that the answer accurately reflects the information in the retrieved context, free from distortions or fabrications that might misrepresent the source material.
3.  [Context Relevancy](https://docs.llamaindex.ai/en/latest/examples/evaluation/relevancy_eval.html): This measures the relevance of the retrieved context and the resulting answer to the original query. The goal is to ensure the system retrieves relevant information to the user’s request.
4.  [Guideline Adherence](https://docs.llamaindex.ai/en/latest/examples/evaluation/guideline_eval.html): This metric determines if the predicted answer follows established guidelines. It checks whether the response meets predefined criteria, including stylistic, factual, and ethical standards, ensuring the answer responds to the query and adheres to specific norms.
5.  [Embedding Semantic Similarity](https://docs.llamaindex.ai/en/latest/examples/evaluation/semantic_similarity_eval.html): This involves calculating a similarity score between the generated and reference answers’ embeddings. It requires reference labels and helps estimate the closeness of the generated response to a reference in terms of semantic content.

💡You can find the documentation pages for the above metrics at [towardsai.net/book.](http://towardsai.net/book)

The evaluation of RAG applications begins with a focus on their primary objective: generate useful outputs supported by contextually relevant facts obtained from retrievers. This analysis then zooms in on specific evaluation metrics such as faithfulness, answer relevancy, and the  [Sensibleness and Specificity Average (SSA)](https://arxiv.org/abs/2001.09977). Google’s SSA metric, which assesses open-domain chatbot responses, evaluates sensibleness (contextual coherence) and specificity (providing detailed and direct responses). Initially involving human evaluators, this metric ensures that outputs are comprehensive and not excessively vague or general.

A high faithfulness score does not necessarily correlate with high relevance. For instance, a response that accurately mirrors the context but does not directly address the query would receive a lower score in answer relevance. This could happen mainly if the response contains incomplete or redundant elements, indicating a gap between the accuracy of the provided context and the direct relevance to the question.

This section proceeds by showing how to compute the faithfulness score using the `FaithfulnessEvaluator` class from LlamaIndex. This metric focuses on preventing the issue of “hallucination” and examines responses to determine their alignment with the retrieved context. It assesses whether the response is consistent with the context and the initial query and fits with reference answers or guidelines. The output of this evaluation is a boolean value, indicating whether the response has successfully met the criteria for accuracy and faithfulness.

To use the FaithfulnessEvaluator, install the required libraries using Python’s package manager using !pip install -q llama-index==0.9.14.post3 deeplake==3.8.12 openai==1.3.8 cohere==4.37.

Following this, set the API keys for OpenAI and Activeloop and replace the placeholders in the provided code with their respective API keys.

In [ ]:
import os
from advanced_rag_custom_utils.helper import get_openai_api_key, get_activeloop_api_key, get_cohere_api_key
OPENAI_API_KEY = get_openai_api_key()
ACTIVELOOP_API_KEY = get_activeloop_api_key()
COHERE_API_KEY = get_cohere_api_key()

Here’s an example for evaluating a single response for faithfulness:

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI

# build service context
llm = OpenAI(model="gpt-4-turbo", temperature=0.0)
service_context = ServiceContext.from_defaults(llm=llm)

from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

vector_store = DeepLakeVectorStore(
dataset_path="hub://genai360/LlamaIndex_paulgraham_essay", overwrite=False
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

from llama_index.evaluation import FaithfulnessEvaluator

# define evaluator
evaluator = FaithfulnessEvaluator(service_context=service_context)

# query index
query_engine = index.as_query_engine()
response = query_engine.query(
    "What does Paul Graham do?"
)

eval_result = evaluator.evaluate_response(response=response)

print("> response:", response)
print("> evaluator result:", eval_result.passing)

> response: Paul Graham is involved in various activities. He is a writer and has given talks on topics such as starting a startup. He has also worked on software development, including creating software for generating websites and building online stores. Additionally, he has been a studio assistant for a beloved teacher who is a painter.  
> evaluator result: True

Most of the above code was previously discussed and will likely be familiar. It includes creating an index from the Deep Lake vector database, using it to make queries to the LLM, and performing the evaluation procedure. In this process, the query engine answers the question and sends its response to the evaluator for further examination.

In this case, the code creates a `FaithfulnessEvaluator` object, a mechanism for evaluating the precision of responses produced by the language model, GPT-4 Turbo. This evaluator operates using the previously defined `service_context`, which contains the configured GPT-4 Turbo model and provides the settings and parameters required for the language model’s optimal functioning.

The fundamental responsibility of the `FaithfulnessEvaluator` is assessing the extent to which the responses from the language model align with accurate and reliable information. It uses a series of criteria or algorithms to accomplish this, comparing the model-generated responses with verified factual data or anticipated results.

The evaluator proceeds to examine the response for its commitment to factual accuracy. This involves determining if the response accurately and dependably represents historical facts related to the query. The outcome of this assessment (`eval_result`) is then reviewed to ascertain if it aligns with the accuracy benchmarks established by the evaluator, as denoted by `eval_result.passing`. The result returns a Boolean (here `True`) value indicating whether the response passed the accuracy and faithfulness checks.